In [1]:
# https://www.kaggle.com/supersp1234/best-single-model-lb-0-1400
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:
DATASET_PATH = "./input/"
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm_notebook
import seaborn as sns

import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
import random
import PIL
from PIL import Image, ImageOps
import math
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import time
import copy

In [3]:
# what is for?
def iso(arr):
    arr = np.array(arr)
    p = np.reshape(arr, [75,75]) > (np.mean(arr) + 2*np.std(arr))
    return p * np.reshape(arr, [75,75])
def size(arr):
    return float(np.sum(arr<-4))/(75*75) 
    #return float(np.sum(arr<-5))/(75*75)

In [4]:
def make_band_data(dataset):
    result = []
    for idx,im in enumerate(dataset):
        tmp = np.array(im).reshape(75,75)
        result.append(tmp)
    return np.array(result)

In [5]:
def make_full_img(X_band1,X_band2):
    return np.array(np.stack([X_band1,X_band2,(X_band1+X_band2)/2],axis=1))

In [6]:
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])
def size(arr):     
    return float(np.sum(arr<-5))/(75*75)

In [7]:
def toTransform(img):
    img = img.astype(float)/255
    tensor = torch.from_numpy(img.copy())
    return tensor
def clip_img(arr,mx):
    return np.clip(arr, (1-mx),mx)

In [8]:
# visualization
def visualization(idx = 10):
    img = train.loc[idx,'band_1']
    plt.imshow(img)
    title = str()
    if train.loc[idx,'is_iceberg'] == 0:
        title = "is Ship"
    else :
        title = "is ICEBERG"
    
    plt.title(title)
    plt.show()
#visualization()

In [9]:
# VGG19 = models.vgg19_bn(pretrained=True)

In [10]:
class read_data(Dataset):
    def __init__(self, data, labels, angle,size=(3,75,75), transform=None):
        self.data =data
        self.labels = labels
        self.transform = transform
        self.angle = angle
        self.size = size
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        sample = {'image' : self.data[idx], 'labels': np.asarray([self.labels[idx]]), 
                  'angle': np.asarray([self.angle[idx]]),'size': np.asarray(self.size[idx])}
        if self.transform :
            sample = self.transform(sample)
            
        return sample


In [11]:
class ToTensor(object):
    def __call__(self, sample):
        image, labels, angle,size = sample['image'], sample['labels'], sample['angle'], sample['size']
        image = image.astype(float)/255
        return {'image': torch.from_numpy(image.copy()).float(),
                'labels': torch.from_numpy(labels).long(),
                'angle': torch.from_numpy(angle).float(),
                'size': torch.from_numpy(size).float()}
class RandomHorizontalFlip(object):
    """Horizontally flip the given PIL.Image randomly with a probability of 0.5."""

    def __call__(self, sample):
        image, labels, angle,size = sample['image'], sample['labels'], sample['angle'],sample['size']

        
        if random.random() < 0.5:
            image=np.flip(image,1)
        
        return {'image': image, 'labels': labels,'angle':angle,'size':size}

class RandomVerticallFlip(object):
    def __call__(self, sample):
        image, labels, angle,size = sample['image'], sample['labels'], sample['angle'],sample['size']
        
        if random.random() < 0.5:
            image=np.flip(image,0)
        
        return {'image': image, 'labels': labels, 'angle':angle,'size':size}   

In [12]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [13]:
from tqdm import tqdm
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = model.state_dict()
    best_loss = 10000.0
    best_acc = 0.0
    running_loss = 0.0
    for epoch in range(1,num_epochs+1):
        print('Epoch {}/{}'.format(epoch,num_epochs))
        print('-'*10)
        
        for phase in ['train','val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
                dataloader=train_loader
                dataset_sizes = len(train_dataset)
            if phase == 'val':
                model.train(False)
                dataloader=val_loader
                dataset_sizes = len(val_dataset)
            running_loss =0.0
            running_corrects = 0
            
            for data in dataloader:
                inputs, labels, angle, size = data['image'], data['labels'], data['angle'], data['size']
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                    angle = Variable(angle.cuda())
                    size = Variable(size.cuda())
                else:
                    inputs, labels, angle = Variable(inputs), Variable(labels), Variable(angle)
                # zero the parameter gradients
                optimizer.zero_grad()
                
                outputs = model(inputs.float(),size.float())
                loss = criterion(outputs.float(), labels.resize((len(labels))).long())
                
                _,preds = torch.max(outputs.data,1)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                running_loss += loss.item() * len(labels)
                running_corrects += torch.sum(preds == labels.resize((len(labels))).long())
            epoch_loss = running_loss / dataset_sizes
            epoch_acc = float(running_corrects) / dataset_sizes
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
               phase, epoch_loss, epoch_acc))
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f} Best val acc: {:4f}'.format(best_loss,best_acc))
    
    model.load_state_dict(best_model_wts)
    return model,best_loss,best_acc


def predict(input_model, loader, dataset, phase):
    dataloader = loader
    dataset_sizes = len(dataset)
    running_loss=0.0
    running_corrects =0
    test_score= []
    
    for data in dataloader:
        inputs, labels, angle,size = data['image'], data['labels'], data['angle'],data['size']
        
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            angle = Variable(angle.cuda())
            size = Variable(size.cuda())

        # forward
        outputs = input_model(inputs.float(),size.float())
        
        if phase == 'val':
            loss = criterion(outputs.float(), labels.resize((len(labels))).long())
            _, preds = torch.max(outputs.data, 1) #for classification
    
            # statistics
            running_loss += loss.item()*len(labels)
            running_corrects += torch.sum(preds == labels.resize((len(labels))).long().data)
        m=nn.Softmax()
        probs=m(outputs)
        probs_value=probs.cpu().data.numpy()
        test_score.extend(probs_value[:,1])
    score=np.array(test_score)
    if phase=='val':
        epoch_loss = running_loss / dataset_sizes
        epoch_acc = float(running_corrects) / dataset_sizes
    
        print('Loss: {:.4f} Acc: {:.4f}'.format(
            epoch_loss, epoch_acc))
        return score,epoch_loss
    else:
        return score

In [14]:
class Vgg11bnNet(nn.Module):
    def __init__(self, num_classes =2):
        super(Vgg11bnNet, self).__init__()
        self.features = nn.Sequential(*list(model_vgg.features.children())) # 28
        self.classifier = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Conv2d(512,512,3,1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.Conv2d(512,128, 3,1, padding=1),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Linear(129,2)
        self.dropout=nn.Dropout()
        self._initialize_weights()
        
    def forward(self,x, size):
        x = self.features(x)
        x = self.classifier(x)
        r = x.size(3)
        x = F.avg_pool2d(x,r)
        x = x.view(x.size(0), -1)
        size = size.reshape(size.shape[0],1)
        x = torch.cat((x,size),1)
        x = self.fc(F.relu(x))
        return x
    
    def _initialize_weights(self):
        for m in self.classifier.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()  
                

In [15]:
warnings.simplefilter("ignore", UserWarning)
n_fold=13
from sklearn.model_selection import KFold
kf = KFold(n_splits=n_fold,shuffle=True)
i=0
train_error = []
val_error = []
data = pd.read_json(DATASET_PATH+"train.json")
test = pd.read_json(DATASET_PATH+"test.json")
data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
data['iso1'] = data['band_1'].apply(iso)
data['iso2'] = data['band_2'].apply(iso)
test['iso1'] = test['band_1'].apply(iso)
test['iso2'] = test['band_2'].apply(iso)
data['s1'] = data['iso1'].apply(size)
data['s2'] = data['iso2'].apply(size)
test['s1'] = test['iso1'].apply(size)
test['s2'] = test['iso2'].apply(size)
data["band_1"] = data["band_1"].apply(lambda x : np.array(x).reshape(75,75))
data["band_2"] = data["band_2"].apply(lambda x : np.array(x).reshape(75,75))
test["band_1"] = test["band_1"].apply(lambda x : np.array(x).reshape(75,75))
test["band_2"] = test["band_2"].apply(lambda x : np.array(x).reshape(75,75))
n_th = 0


In [16]:

for train_ind, val_ind in kf.split(data):
    n_th +=1
    print("============== Ksplit phase %d/%d==========="%(n_th,n_fold))
    train = data.loc[train_ind]
    val = data.loc[val_ind]
    
    train=train.reset_index()
    del train['index']
    val=val.reset_index()
    del val['index']
    
    band_1_tr = make_band_data(train['band_1'])
    band_2_tr = make_band_data(train['band_2'])
    full_img_tr = make_full_img(band_1_tr,band_2_tr)
    angle_tr = train['inc_angle']
    size_tr = train['s1']
    band_1_val = make_band_data(val['band_1'])
    band_2_val = make_band_data(val['band_2'])
    full_img_val = make_full_img(band_1_val,band_2_val)
    angle_val = val['inc_angle']
    size_val = val['s1']     
    
    train_dataset = read_data(data=full_img_tr, labels=train['is_iceberg'],
                                      angle=angle_tr,size=size_tr,
                                             transform=transforms.Compose([
                                                   
                                                   RandomHorizontalFlip(),
                                                   RandomVerticallFlip(),
                                                   ToTensor(),

                                               ]))
    train_loader = DataLoader(dataset=train_dataset, batch_size=128,
                              shuffle=True, num_workers=4)  
    
    val_dataset = read_data(data=full_img_val, labels=val['is_iceberg'],
                                angle=angle_val,size=size_val,
                                             transform=transforms.Compose([
                                                   ToTensor(),
                                               ]))
    val_loader = DataLoader(dataset=val_dataset, batch_size=8,
                            shuffle=False, num_workers=4)  


    model_vgg = models.vgg19_bn(pretrained=True)
    model_ft=Vgg11bnNet()
    
    op_parameters = filter(lambda p: p.requires_grad, model_ft.parameters())
    
    if use_gpu:
        model_ft = model_ft.cuda()
    
    criterion = nn.CrossEntropyLoss()
    
    optimizer_conv = optim.Adam(op_parameters, lr=0.001, betas=(0.9, 0.99))
    exp_lr_scheduler1 = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
    
    new_model,best_loss,best_acc = train_model(model_ft, criterion,optimizer_conv,
                             exp_lr_scheduler1, num_epochs=40)
    
    snapshot_path='./weight/vgg19bnfcn_10cv1/'
    if not os.path.exists(snapshot_path):
        os.makedirs(snapshot_path) 
    
    # torch.save(model_ft, snapshot_path+'vggbnw_fcn_50_%d_%.4f_%.4f.pkl' % (i,best_loss,best_acc)) 
    
    val_result=predict(new_model,val_loader,val_dataset,'val')
    assert(val_result[1]==best_loss)
    test['is_iceberg'] = 0
    angle_test=test['inc_angle']
    size_test = test['s1']
    full_img_test = make_full_img(make_band_data(test['band_1']),make_band_data(test['band_2']))
    test_dataset = read_data(data=full_img_test, labels=test['is_iceberg'],angle=angle_test,size=size_test,
                                             transform=transforms.Compose([
                                                   ToTensor(),
                                               ]))
    
    
    test_loader = DataLoader(dataset=test_dataset, batch_size=8,
                             shuffle=False, num_workers=4) 
    test_result=predict(new_model,test_loader,test_dataset,'test')
 
    test_id=test['id']   
    truth=pd.DataFrame(test_result, columns=['is_iceberg'])
    frame=[test_id,truth]
    result=pd.concat(frame,axis=1)
    
    snapshot_csv='./result/vgg19bnfcn_10cv1/'
    if not os.path.exists(snapshot_csv):
        os.makedirs(snapshot_csv) 
    output=snapshot_csv+'vggbnw_fcn_%d.csv' %i
    result.to_csv(output,index=False) 

    i=i+1

============== Ksplit phase 1/13===========
Epoch 1/40
----------
train Loss: 0.5159 Acc: 0.7507
val Loss: 0.6134 Acc: 0.5484
Epoch 2/40
----------
train Loss: 0.2783 Acc: 0.8851
val Loss: 0.8491 Acc: 0.5484
Epoch 3/40
----------
train Loss: 0.2360 Acc: 0.9007
val Loss: 1.6814 Acc: 0.6129
Epoch 4/40
----------
train Loss: 0.1793 Acc: 0.9264
val Loss: 1.7504 Acc: 0.6048
Epoch 5/40
----------
train Loss: 0.1565 Acc: 0.9358
val Loss: 4.0359 Acc: 0.5081
Epoch 6/40
----------
train Loss: 0.1337 Acc: 0.9527
val Loss: 1.0651 Acc: 0.7097
Epoch 7/40
----------
train Loss: 0.0852 Acc: 0.9703
val Loss: 0.2238 Acc: 0.9355
Epoch 8/40
----------
train Loss: 0.0745 Acc: 0.9736
val Loss: 0.2112 Acc: 0.9435
Epoch 9/40
----------
train Loss: 0.0510 Acc: 0.9818
val Loss: 0.2181 Acc: 0.9355
Epoch 10/40
----------
train Loss: 0.0339 Acc: 0.9905
val Loss: 0.2239 Acc: 0.9274
Epoch 11/40
----------
train Loss: 0.0260 Acc: 0.9899
val Loss: 0.2355 Acc: 0.9435
Epoch 12/40
----------
train Loss: 0.0274 Acc: 0.991

In [17]:
avg_data = pd.read_csv("result/vgg19bnfcn_10cv1/vggbnw_fcn_0.csv")
avg_data["is_iceberg"] = avg_data["is_iceberg"]/10
print(avg_data.iloc[1]["is_iceberg"])
for i in range(1,10):
    filename = "result/vgg19bnfcn_10cv1/vggbnw_fcn_" + str(i) + ".csv"
    data = pd.read_csv(filename)
    avg_data['is_iceberg'] = avg_data['is_iceberg'] + data['is_iceberg']/10

0.09476293999999999


In [18]:
avg_data

,id,is_iceberg
0,5941774d,0.019820
1,4023181e,0.600440
2,b20200e4,0.015500
3,e7f018bb,0.998829
4,4371c8c3,0.551448
...,...,...
8419,16ee9b50,0.000082
8420,5a599eb7,0.153961
8421,df30d6dd,0.015733
8422,18af95b1,0.997771


In [19]:
avg_data.to_csv("result/epoch40.csv",index=False)

In [20]:
tmp = pd.read_csv("./result/epoch40.csv")

In [21]:
tmp

,id,is_iceberg
0,5941774d,0.019820
1,4023181e,0.600440
2,b20200e4,0.015500
3,e7f018bb,0.998829
4,4371c8c3,0.551448
...,...,...
8419,16ee9b50,0.000082
8420,5a599eb7,0.153961
8421,df30d6dd,0.015733
8422,18af95b1,0.997771


In [22]:
!kaggle competitions submit -c statoil-iceberg-classifier-challenge -f result/epoch40.csv -m "epoch 40"

100%|████████████████████████████████████████| 210k/210k [00:03<00:00, 68.2kB/s]
Successfully submitted to Statoil/C-CORE Iceberg Classifier Challenge